<a href="https://colab.research.google.com/github/zloelias/final-project/blob/master/bert_lenta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install tensorflow_text
! pip install textattack['tensorflow']
! pip install datasets --upgrade

from textattack.constraints.semantics.sentence_encoders.universal_sentence_encoder import UniversalSentenceEncoder

class MultilingualUniversalSentenceEncoder(UniversalSentenceEncoder):

    def __init__(self, threshold=0.8, large=False, metric="angular", **kwargs):
        UniversalSentenceEncoder.__init__(self, threshold=threshold, large=large, metric=metric, **kwargs)
        if large:
            tfhub_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3"
        else:
            tfhub_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"

        self._tfhub_url = tfhub_url



     |████████████████████████████████| 4.9 MB 10.6 MB/s 
     |████████████████████████████████| 373 kB 15.4 MB/s 
     |████████████████████████████████| 60 kB 9.3 MB/s 
     |████████████████████████████████| 322 kB 75.2 MB/s 
     |████████████████████████████████| 69 kB 9.0 MB/s 
     |████████████████████████████████| 298 kB 86.5 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 3.3 MB 63.5 MB/s 
     |████████████████████████████████| 769 kB 72.1 MB/s 
     |████████████████████████████████| 124 kB 85.1 MB/s 
     |████████████████████████████████| 454.3 MB 12 kB/s 
     |████████████████████████████████| 462 kB 79.5 MB/s 
     |████████████████████████████████| 4.0 MB 53.7 MB/s 
     |████████████████████████████████| 1.2 MB 68.9 MB/s 
     |████████████████████████████████| 4.9 MB 77.1 MB/s 
     |████████████████████████████████| 4.4 MB 97.3 MB/s 
     |████████████████████████████████| 4.3 MB 76.6 MB/s 
     |████████████

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:17<00:00, 27.1MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpe5chkpjy.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [3]:
# При установке из репозитория при работе не подцепляются другие модели, работает только дефолтный bert-base-uncased

#! pip install tensorflow_text
#! git clone https://github.com/zloelias/TextAttack.git
#! pip install /content/TextAttack[tensorflow] --use-feature=in-tree-build
#! pip install datasets --upgrade
#! pip install captum

In [4]:
import torch
from copy import deepcopy

from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer

#from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerDeepLiftShap, InternalInfluence, LayerGradientXActivation
#from captum.attr import visualization as viz

import nltk
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else: 
    device = torch.device("cpu")
    
print(device)

cuda:0


In [6]:
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder#, MultilingualUniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapMaskedLM, CompositeTransformation

from textattack.shared.utils import LazyLoader

from textattack.constraints.pre_transformation.max_word_index_modification import MaxWordIndexModification
from textattack.constraints.pre_transformation.max_modification_rate import MaxModificationRate

from textattack.attack_recipes.attack_recipe import AttackRecipe


In [7]:
class BAEGarg2019RU(AttackRecipe):
    """Siddhant Garg and Goutham Ramakrishnan, 2019.

    BAE: BERT-based Adversarial Examples for Text Classification.

    https://arxiv.org/pdf/2004.01970

    This is "attack mode" 1 from the paper, BAE-R, word replacement.

    We present 4 attack modes for BAE based on the
        R and I operations, where for each token t in S:
        • BAE-R: Replace token t (See Algorithm 1)
        • BAE-I: Insert a token to the left or right of t
        • BAE-R/I: Either replace token t or insert a
        token to the left or right of t
        • BAE-R+I: First replace token t, then insert a
        token to the left or right of t
    """

    @staticmethod
    def build(model_wrapper, max_rate, max_seq_len, language, lan, la, base_model_name):
        # "In this paper, we present a simple yet novel technique: BAE (BERT-based
        # Adversarial Examples), which uses a language model (LM) for token
        # replacement to best fit the overall context. We perturb an input sentence
        # by either replacing a token or inserting a new token in the sentence, by
        # means of masking a part of the input and using a LM to fill in the mask."
        #
        # We only consider the top K=50 synonyms from the MLM predictions.
        #
        # [from email correspondance with the author]
        # "When choosing the top-K candidates from the BERT masked LM, we filter out
        # the sub-words and only retain the whole words (by checking if they are
        # present in the GloVE vocabulary)"
        #
        transformation = CompositeTransformation(
            [
                WordSwapMaskedLM(masked_language_model=base_model_name, method="bae", max_candidates=20, max_length=max_seq_len),
            ]
        )
        #
        # Don't modify the same word twice or stopwords.
        #
        constraints = [RepeatModification(), StopwordModification(language=language)]

        # For the R operations we add an additional check for
        # grammatical correctness of the generated adversarial example by filtering
        # out predicted tokens that do not form the same part of speech (POS) as the
        # original token t_i in the sentence.
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True,
            language_nltk=lan,
            language_stanza=la))

        # "To ensure semantic similarity on introducing perturbations in the input
        # text, we filter the set of top-K masked tokens (K is a pre-defined
        # constant) predicted by BERT-MLM using a Universal Sentence Encoder (USE)
        # (Cer et al., 2018)-based sentence similarity scorer."
        #
        # "[We] set a threshold of 0.8 for the cosine similarity between USE-based
        # embeddings of the adversarial and input text."
        #
        # [from email correspondence with the author]
        # "For a fair comparison of the benefits of using a BERT-MLM in our paper,
        # we retained the majority of TextFooler's specifications. Thus we:
        # 1. Use the USE for comparison within a window of size 15 around the word
        # being replaced/inserted.
        # 2. Set the similarity score threshold to 0.1 for inputs shorter than the
        # window size (this translates roughly to almost always accepting the new text).
        # 3. Perform the USE similarity thresholding of 0.8 with respect to the text
        # just before the replacement/insertion and not the original text (For
        # example: at the 3rd R/I operation, we compute the USE score on a window
        # of size 15 of the text obtained after the first 2 R/I operations and not
        # the original text).
        # ...
        # To address point (3) from above, compare the USE with the original text
        # at each iteration instead of the current one (While doing this change
        # for the R-operation is trivial, doing it for the I-operation with the
        # window based USE comparison might be more involved)."
        #
        # Finally, since the BAE code is based on the TextFooler code, we need to
        # adjust the threshold to account for the missing / pi in the cosine
        # similarity comparison. So the final threshold is 1 - (1 - 0.8) / pi
        # = 1 - (0.2 / pi) = 0.936338023.
        use_constraint = MultilingualUniversalSentenceEncoder(
            threshold=0.936338023,
            metric="cosine",
            compare_against_original=True,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)

        constraints.append(MaxWordIndexModification(max_seq_len))
        constraints.append(MaxModificationRate(max_rate, min_threshold=1))

        #
        # Goal is untargeted classification.
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # "We estimate the token importance Ii of each token
        # t_i ∈ S = [t1, . . . , tn], by deleting ti from S and computing the
        # decrease in probability of predicting the correct label y, similar
        # to (Jin et al., 2019).
        #
        # • "If there are multiple tokens can cause C to misclassify S when they
        # replace the mask, we choose the token which makes Sadv most similar to
        # the original S based on the USE score."
        # • "If no token causes misclassification, we choose the perturbation that
        # decreases the prediction probability P(C(Sadv)=y) the most."
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return BAEGarg2019RU(goal_function, constraints, transformation, search_method)


In [8]:
import textattack
import random
import numpy as np

experiments = {
    'bae_same_model': [
#        {
#            'dataset': {'name': 'kinopoisk', 'url': 'zloelias/kinopoisk-reviews'},
#            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-kinopoisk-reviews-finetuned-clf'),
#            'language': ('ru', 'rus', 'russian'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 312,
#            'columns': (['text'], 'labels'),
#        },
        {
            'dataset': {'name': 'lenta', 'url': 'zloelias/lenta-ru'},
            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-lenta-ru-finetuned-clf'),
            'language': ('ru', 'rus', 'russian'),
            'max_rate': np.linspace(0.1, 0.9, 9),
            'max_seq_len': 312,
            'columns': (['text'], 'labels'),
        },
#        {
#            'dataset': {'name': 'ag_news', 'url': 'ag_news'},
#            'model': ('bert-base-uncased', 'textattack/bert-base-uncased-ag-news'),
#            'language': ('en', 'eng', 'english'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 512,
#            'columns': (['text'], 'label'),
#        },
#        {
#            'dataset': {'name': 'rotten_tomatoes', 'url': 'rotten_tomatoes'},
#            'model': ('bert-base-uncased', 'textattack/bert-base-uncased-rotten-tomatoes'),
#            'language': ('en', 'eng', 'english'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 512,
#            'columns': (['text'], 'label'),
#        },
    ]
}

for attack_name in experiments.keys():
    for attack_data in experiments[attack_name]:
        dataset_name, dataset_url = [attack_data['dataset'][k] for k in attack_data['dataset'].keys()]
        dataset_columns = attack_data['columns']
        dataset = HuggingFaceDataset(dataset_url, None, 'test', dataset_columns=dataset_columns)

        base_model_name, model_name = attack_data['model']
        original_model = AutoModelForSequenceClassification.from_pretrained(model_name)
        original_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        model = HuggingFaceModelWrapper(original_model,original_tokenizer)

        la, lan, language = attack_data['language']
        for max_rate in attack_data['max_rate']:
            experiment_result_dir = f'/content/drive/MyDrive/Colab Notebooks/final-project/experiments/{attack_name}/{dataset_name}/{max_rate}/'
            print(experiment_result_dir)
            attack = BAEGarg2019RU.build(model, max_rate, attack_data['max_seq_len'], language, lan, la, base_model_name)

            attack_args = textattack.AttackArgs(
                num_examples=100, 
                log_to_csv=experiment_result_dir + "log.csv", 
                checkpoint_interval=50, 
                checkpoint_dir=experiment_result_dir + "checkpoints", 
                disable_stdout=True
            )
            attacker = textattack.Attacker(attack, dataset, attack_args)
            attacker.attack_dataset()
        #print(attack_data)
        #print(model_name, model_url)

Using custom data configuration zloelias___lenta-ru-dc5084b443d90e4e


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/240M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.7M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path /content/drive/MyDrive/Colab Notebooks/final-project/experiments/bae_same_

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapMaskedLM(
        (method):  bae
        (masked_lm_name):  BertForMaskedLM
        (max_length):  312
        (max_candidates):  20
        (min_confidence):  0.0005
      )
    )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): MultilingualUniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
    (4): MaxWordIndexModification(
        (max_length):  312
      )
    (5): MaxModificationRate(
        (max_rate):  0.9
        (min_threshold):  1
      

[Succeeded / Failed / Skipped / Total] 7 / 43 / 0 / 50:  50%|█████     | 50/100 [35:25<35:25, 42.51s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/bae_same_model/lenta/0.9/checkpoints/1639617776335.ta.chkpt" at 2021-12-16 01:22:56 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 80 / 1 / 100: 100%|██████████| 100/100 [1:04:32<00:00, 38.72s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/bae_same_model/lenta/0.9/checkpoints/1639619522861.ta.chkpt" at 2021-12-16 01:52:02 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 19 / 80 / 1 / 100: 100%|██████████| 100/100 [1:04:32<00:00, 38.72s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 80     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 99.0%  |
| Accuracy under attack:        | 80.0%  |
| Attack success rate:          | 19.19% |
| Average perturbed word %:     | 11.8%  |
| Average num. words per input: | 195.18 |
| Avg num queries:              | 808.38 |
+-------------------------------+--------+
